# Using E3 in a Jupyter Notebook

As an API, E3 is designed to be callable from any programming language capable of making API requests. This example will illustrate the process using Python and assumes you already have Python installed. 

**If running locally**, the **requests** library will be installed to access the API. This can be done using the following command in your computer's terminal/command prompt provided you have Python already installed (if **requests** is already installed you can skip this step).

In [ ]:
%pip install -q requests

**If running Jupyter from the browser (JupyterLite)** you will need to run the following commands. Note that requests doesn't work in JupyterLite.

In [ ]:
from pyodide.ffi import to_js
from IPython.display import JSON
from js import Object, fetch

With requests installed the only items remaining are to construct the JSON string to send to the API and to properly construct the API request in python. This notebook will cover constructing the JSON string from scratch. For examples of how to use input from CSV, XML and XLS/XLSX files see the **E3 Example CSV**, **E3 Example XML** and **E3 Example Excel** (in development) files, respectively. 

## Constructing the JSON string

The format for the JSON request must be correct, otherwise E3 will return an error. In this section we will look at the required format for individual objects within E3 before putting all of them into a single JSON string for an API request. The discussion here will be focused on formatting, for a more in depth look please see the technical manual for E3. We'll start with the **Analysis** object format. **Note:** If you double click on some cells you may see **\<br>** and **\&emsp;** tags. These are used exclusively for formatting in Jupyter and **should not appear in the input JSON string sent to the E3 API.**

"analysisObject" : { <br>
&emsp;"type" : "BCA", <br>
&emsp;"projectType" : "Infrastructure", <br>
&emsp;"outputObjects" : [ <br>
&emsp;&emsp;"required", <br>
&emsp;&emsp;"measure", <br>
&emsp;&emsp;"optional" <br>
&emsp;], <br>
&emsp;"studyPeriod" : 50, <br>
&emsp;"timestepValue" : "Year", <br>
&emsp;"timestepComp" : "End_Of_Year", <br>
&emsp;"outputReal" : true, <br>
&emsp;"interestRate": 0.03, <br>
&emsp;"discountRateReal" : 0.03, <br>
&emsp;"inflationRate" : 0.023, <br>
&emsp;"marr" : 0.03, <br>
&emsp;"reinvestRate" : 0.03, <br>
&emsp;"federalIncomeRate": 0.22, <br>
&emsp;"otherIncomeRate": 0.05, <br>
&emsp;"numberOfAlternatives" : 2, <br>
&emsp;"baseAlternative" : 0 <br>
}

There should only be one Analysis object for the input. **Alternative** objects have the following basic format.

{ <br>
&emsp;"id" : 0, <br>
&emsp;"name" : "Alt 0 Status Quo", <br>
&emsp;"bcns" : [ <br>
&emsp;&emsp;0 <br>
&emsp;] <br>
}

Multiple alternatives are required for analysis since E3 assumes comparison to a baseline. If only one alternative exists in an analysis you can simply define an alternative with a single cost object with zero quant or quantValue (See the examples for BCNs. An example with two alternatives is presented here. Note that the **alternativeObjects** key is now prepended and the  associated value is now a list of alternative objects.

"alternativeObjects" : [ <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 0, <br>
&emsp;&emsp;"name" : "Alt 0 Status Quo", <br>
&emsp;&emsp;"bcns" : [ <br>
&emsp;&emsp;&emsp;0 <br>
&emsp;&emsp;] <br>
&emsp;}, <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 1, <br>
&emsp;&emsp;"name" : "Alt 1 Build Monument", <br>
&emsp;&emsp;"bcns" : [ <br>
&emsp;&emsp;&emsp;1, <br>
&emsp;&emsp;&emsp;2, <br>
&emsp;&emsp;&emsp;3, <br>
&emsp;&emsp;&emsp;4 <br>
&emsp;&emsp;] <br>
&emsp;} <br>
]

An individual **BCN** object has the following format if the BCN doesn't recur.

{ <br>
&emsp;"id" : 0, <br>
&emsp;"altIds" : [ <br>
&emsp;&emsp;0 <br>
&emsp;], <br>
&emsp;"type" : "Cost", <br>
&emsp;"subType" : "Direct", <br>
&emsp;"name" : "Investment Cost - Status Quo", <br>
&emsp;"tags" : "Initial Investment", <br>
&emsp;"initialOccurrence" : 0, <br>
&emsp;"real" : true, <br>
&emsp;"invest" : true, <br>
&emsp;"residualValue" : false, <br>
&emsp;"residualValueOnly" : false, <br>
&emsp;"quantityValue" : 0.0, <br>
&emsp;"quantity" : 1.0 <br>
}

If the **BCN** does recur it has the following general format.

{ <br>
&emsp;"id" : 2, <br>
&emsp;"altIds" : [ <br>
&emsp;&emsp;1 <br>
&emsp;], <br>
&emsp;"type" : "Cost", <br>
&emsp;"subType" : "Direct", <br>
&emsp;"name" : "Maintain Monument", <br>
&emsp;"tags" : "OMR Costs", <br>
&emsp;"initialOccurrence" : 1, <br>
&emsp;"real" : true, <br>
&emsp;"invest" : false, <br>
&emsp;"life" : 50, <br>
&emsp;"residualValue" : false, <br>
&emsp;"residualValueOnly" : false, <br>
&emsp;"recur" : { <br>
&emsp;&emsp;"interval" : 1, <br>
&emsp;&emsp;"end" : 50, <br>
&emsp;&emsp;"varRate" : "Percent_Delta", <br>
&emsp;&emsp;"varValue" : 0.0 <br>
&emsp;}, <br>
&emsp;"quantityValue" : 1000.0, <br>
&emsp;"quantity" : 1.0 <br>
}

Now putting all **BCN** objects together.

"bcnObjects" : [ <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 0, <br>
&emsp;&emsp;"altIds" : [ <br>
&emsp;&emsp;&emsp;0 <br>
&emsp;&emsp;], <br>
&emsp;&emsp;"type" : "Cost", <br>
&emsp;&emsp;"subType" : "Direct", <br>
&emsp;&emsp;"name" : "Investment Cost - Status Quo", <br>
&emsp;&emsp;"tags" : "Initial Investment", <br>
&emsp;&emsp;"initialOccurrence" : 0, <br>
&emsp;&emsp;"real" : true, <br>
&emsp;&emsp;"invest" : true, <br>
&emsp;&emsp;"residualValue" : false, <br>
&emsp;&emsp;"residualValueOnly" : false, <br>
&emsp;&emsp;"quantityValue" : 0.0, <br>
&emsp;&emsp;"quantity" : 1.0 <br>
&emsp;}, <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 1, <br>
&emsp;&emsp;"altIds" : [ <br>
&emsp;&emsp;&emsp;1 <br>
&emsp;&emsp;], <br>
&emsp;&emsp;"type" : "Cost", <br>
&emsp;&emsp;"subType" : "Direct", <br>
&emsp;&emsp;"name" : "Construct Monument", <br>
&emsp;&emsp;"tags" : "Investment Cost", <br>
&emsp;&emsp;"initialOccurrence" : 0, <br>
&emsp;&emsp;"real" : true, <br>
&emsp;&emsp;"invest" : true, <br>
&emsp;&emsp;"residualValue" : false, <br>
&emsp;&emsp;"residualValueOnly" : false, <br>
&emsp;&emsp;"quantityValue" : 100000.0, <br>
&emsp;&emsp;"quantity" : 1.0 <br>
&emsp;}, <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 2, <br>
&emsp;&emsp;"altIds" : [ <br>
&emsp;&emsp;&emsp;1 <br>
&emsp;&emsp;], <br>
&emsp;&emsp;"type" : "Cost", <br>
&emsp;&emsp;"subType" : "Direct", <br>
&emsp;&emsp;"name" : "Maintain Monument", <br>
&emsp;&emsp;"tags" : "OMR Costs", <br>
&emsp;&emsp;"initialOccurrence" : 1, <br>
&emsp;&emsp;"real" : true, <br>
&emsp;&emsp;"invest" : false, <br>
&emsp;&emsp;"life" : 50, <br>
&emsp;&emsp;"residualValue" : false, <br>
&emsp;&emsp;"residualValueOnly" : false, <br>
&emsp;&emsp;"recur" : { <br>
&emsp;&emsp;&emsp;"interval" : 1, <br>
&emsp;&emsp;&emsp;"end" : 50, <br>
&emsp;&emsp;&emsp;"varRate" : "Percent_Delta", <br>
&emsp;&emsp;&emsp;"varValue" : 0.0 <br>
&emsp;&emsp;}, <br>
&emsp;&emsp;"quantityValue" : 1000.0, <br>
&emsp;&emsp;"quantity" : 1.0 <br>
&emsp;}, <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 3, <br>
&emsp;&emsp;"altIds" : [ <br>
&emsp;&emsp;&emsp;1 <br>
&emsp;&emsp;], <br>
&emsp;&emsp;"type" : "Benefit", <br>
&emsp;&emsp;"subType" : "Direct", <br>
&emsp;&emsp;"name" : "Free Publicity", <br>
&emsp;&emsp;"tags" : "Publicity", <br>
&emsp;&emsp;"initialOccurrence" : 1, <br>
&emsp;&emsp;"real" : true, <br>
&emsp;&emsp;"invest" : false, <br>
&emsp;&emsp;"residualValue" : false, <br>
&emsp;&emsp;"residualValueOnly" : false, <br>
&emsp;&emsp;"quantityValue" : 50000.0, <br>
&emsp;&emsp;"quantity" : 1.0 <br>
&emsp;}, <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 4, <br>
&emsp;&emsp;"altIds" : [ <br>
&emsp;&emsp;&emsp;1 <br>
&emsp;&emsp;], <br>
&emsp;&emsp;"type" : "Benefit", <br>
&emsp;&emsp;"subType" : "Externality", <br>
&emsp;&emsp;"name" : "Increased Tourism", <br>
&emsp;&emsp;"tags" : "Tourism", <br>
&emsp;&emsp;"initialOccurrence" : 1, <br>
&emsp;&emsp;"real" : true, <br>
&emsp;&emsp;"invest" : false, <br>
&emsp;&emsp;"residualValue" : false, <br>
&emsp;&emsp;"residualValueOnly" : false, <br>
&emsp;&emsp;"recur" : { <br>
&emsp;&emsp;&emsp;"interval" : 1 <br>
&emsp;&emsp;}, <br>
&emsp;&emsp;"quantityValue" : 5000.0, <br>
&emsp;&emsp;"quantity" : 1.0 <br>
&emsp;} <br>
]

With all the individual pieces in place the final JSON string can be constructed. Note that the following is a python dictionary, hence the capitalized **True** and **False**. Due to the similar structure of a Python dictionary and a JSON string and the availability of the **JSON** library it is easier to define the input as a dictionary first, which usualy requires only minor edits such as the aforementioned capilatization issue for Booleans, and then allowing Python to handle serialization. This dictionary will be converted to a JSON string using python's JSON library. If you want to work with JSON directly you may simply define **inputJSON** as a string literal with any necessary escape characters and pass the raw string in the API request for the **data** input.

In [ ]:
inputJSON = {
  "analysisObject" : {
    "type" : "BCA",
    "projectType" : "Infrastructure",
    "outputObjects" : [
      "measure"
    ],
    "studyPeriod" : 50,
    "timestepValue" : "Year",
    "timestepComp" : "End_Of_Year",
    "outputReal" : True,
    "intereseRate": 0.03,
    "discountRateReal" : 0.03,
    "inflationRate" : 0.023,
    "marr" : 0.03,
    "reinvestRate" : 0.03,
    "federalIncomeRate": 0.22,
    "otherIncomeRate": 0.05,
    "numberOfAlternatives" : 2,
    "baseAlternative" : 0
  },
  "alternativeObjects" : [
    {
      "id" : 0,
      "name" : "Alt 0 Status Quo",
      "bcns" : [
        0
      ]
    },
    {
      "id" : 1,
      "name" : "Alt 1 Build Monument",
      "bcns" : [
        1,
        2,
        3,
        4
      ]
    }
  ],
  "bcnObjects" : [
    {
      "id" : 0,
      "altIds" : [
        0
      ],
      "type" : "Cost",
      "subType" : "Direct",
      "name" : "Investment Cost - Status Quo",
      "tags" : "Initial Investment",
      "initialOccurrence" : 0,
      "real" : True,
      "invest" : True,
      "residualValue" : False,
      "residualValueOnly" : False,
      "quantityValue" : 0.0,
      "quantity" : 1.0
    },
    {
      "id" : 1,
      "altIds" : [
        1
      ],
      "type" : "Cost",
      "subType" : "Direct",
      "name" : "Construct Monument",
      "tags" : "Investment Cost",
      "initialOccurrence" : 0,
      "real" : True,
      "invest" : True,
      "residualValue" : False,
      "residualValueOnly" : False,
      "quantityValue" : 100000.0,
      "quantity" : 1.0
    },
    {
      "id" : 2,
      "altIds" : [
        1
      ],
      "type" : "Cost",
      "subType" : "Direct",
      "name" : "Maintain Monument",
      "tags" : "OMR Costs",
      "initialOccurrence" : 1,
      "real" : True,
      "invest" : False,
      "life" : 50,
      "residualValue" : False,
      "residualValueOnly" : False,
      "recur" : {
        "interval" : 1,
        "varRate" : "Percent_Delta",
        "varValue" : 0.0
      },
      "quantityValue" : 1000.0,
      "quantity" : 1.0
    },
    {
      "id" : 3,
      "altIds" : [
        1
      ],
      "type" : "Benefit",
      "subType" : "Direct",
      "name" : "Free Publicity",
      "tags" : "Publicity",
      "initialOccurrence" : 1,
      "real" : True,
      "invest" : False,
      "residualValue" : False,
      "residualValueOnly" : False,
      "quantityValue" : 50000.0,
      "quantity" : 1.0
    },
    {
      "id" : 4,
      "altIds" : [
        1
      ],
      "type" : "Benefit",
      "subType" : "Externality",
      "name" : "Increased Tourism",
      "tags" : "Tourism",
      "initialOccurrence" : 1,
      "real" : True,
      "invest" : False,
      "residualValue" : False,
      "residualValueOnly" : False,
      "recur" : {
        "interval" : 1
      },
      "quantityValue" : 5000.0,
      "quantity" : 1.0
    }
  ]
}

An API key is required to use E3. Paste yours where it says API_KEY. To get an API key you need to go to https://e3.nist.gov, create a log in and request a new key. The key should be passed in the **Authorization** header. The **Accept** header defines the output the client will accept while the **Content-Type** header defines the format of the sent data.

In [ ]:
headers = {'Content-Type': "application/json", 
           'Accept': "application/json", 
           'Authorization': "Api-Key: CXizFToEOIWa.a8606a6d-d9ae-4907-93a7-ea64c253c37f"
          }

The full request can now be made. **If running locally use the request library and the following code.**

In [ ]:
import requests
import json
api_url = "https://e3.nist.gov/api/v2/analysis"
response = requests.post(api_url, data=json.dumps(inputJSON), headers=headers)
response.json()
response.status_code

# The status code can be dropped if the request is successful
if response.status_code == 200:
    response = response.json()
    print(response)
else:
    print("Error:", response.status_code)

**If running through JupyterLite use the following code.**

In [ ]:
import json

resp = await fetch("https://e3.nist.gov/api/v2/analysis",
                       method = "POST",
                       body = json.dumps(inputJSON),
                       credentials = "same-origin",
                       headers = Object.fromEntries(to_js(headers))
                      )
response = await resp.text()
response = json.loads(response)

print(response)

We can now pull output from the reponse object.

In [ ]:
print("Net Benefits: Alt 0 Status Quo:", response['measure'][0]['netBenefits'])
print("Net Benefits: Alt 1 Build Monument:", response['measure'][1]['netBenefits'])
print("BCR: Alt 0 Status Quo:", response['measure'][0]['bcr'])
print("BCR: Alt 1 Build Monument:", response['measure'][1]['bcr'])